In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth

In [2]:
import time
from numpy import random

In [3]:
CHROMEDRIVER_PATH = "/executables/chromedriver.exe"
STACKOVERFLOW_LOGIN_URL = "https://stackoverflow.com/users/login"

In [4]:
def sleeprandom(minimum=0.5, maximum=1):
    st = random.rand() * (maximum - minimum) + minimum
    time.sleep(st)

In [5]:
class Webdriver_Handler:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('disable-infobars')
        options.add_experimental_option('useAutomationExtension', False)

        self.timeout = 10       

        service = Service(executable_path=CHROMEDRIVER_PATH)
        self.wd = webdriver.Chrome(service=service, options=options)
        
        self.wd.implicitly_wait(self.timeout)

        stealth(
            self.wd,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )
    
    def login(self, username, password):
        self.wd.get(STACKOVERFLOW_LOGIN_URL)
        sleeprandom()
        self.try_disabling_cookies()
        sleeprandom()
        self.wd.find_element(By.ID, "email").send_keys(username)
        sleeprandom()
        self.wd.find_element(By.ID, "password").send_keys(password)
        sleeprandom()
        self.wd.find_element(By.ID, "submit-button").click()
        sleeprandom()
    
    def try_disabling_cookies(self):
        try:
            self.wd.find_element(By.XPATH, """//button[contains(text(), "Necessary cookies only")]""").click()
            return True
        except:
            return False
    
    def is_logged_in(self):
        try: 
            username = self.wd.find_element(By.XPATH, """//a[@class="s-topbar--item s-user-card s-user-card__small m0 px12 js-gps-track"]/span""").text
            reputation = self.wd.find_element(By.XPATH,"""//li[contains(@title, "reputation")]""").text.split("\n")[0].replace(" ","")
            print(f"Logged in as {username} with {reputation} reputation")
            return True
        except:
            print("Not logged in")
            return False
    
    def upvote_questions(self, num_upvotes=3):
        self.wd.get("https://stackoverflow.com/questions/tagged/python")
        question_links = [a.get_attribute("href") for a in self.wd.find_elements(By.XPATH, """//div[@id="questions"]/div/div/h3/a""")]
        print(question_links)
        sleeprandom()

        upvotes_given = 0
        for question_link in question_links:
            self.wd.get(question_link)
            sleeprandom()
            vote_count = int(self.wd.find_element(By.XPATH, """//div[@id="question"]/div/div/div/div[contains(@class,"vote-count")]""").text.replace(" ",""))
            if vote_count < 0:
                print("Skipping question with negative votes")
                continue

            upvote_button = self.wd.find_element(By.XPATH, """//div[@id="question"]/div/div/div/button[contains(@class,"vote-up")]""")

            if upvote_button.get_attribute("aria-pressed") == "false":
                upvote_button.click()
                upvotes_given += 1
                print(f"Upvoted {question_link}")
                sleeprandom()
            else:
                print("Already upvoted")
            
            if upvotes_given >= num_upvotes:
                break
        
        print(f"Upvoted {upvotes_given} questions")
        
            



